### Nested Context Managers

In [2]:
with open('file1.txt') as f:
    for row in f:
        print(row, end='')
print('\n--------------------------')
with open('file2.txt') as f:
    for row in f:
        print(row, end='')
print('\n--------------------------')
with open('file3.txt') as f:
    for row in f:
        print(row, end='')
print('\n--------------------------')

file1_line1
file1_line2
file1_line3
--------------------------
file2_line1
file2_line2
file2_line3
--------------------------
file3_line1
file3_line2
file3_line3
--------------------------


In [4]:
with open('file1.txt') as f1, open('file2.txt') as f2:
    print(f1.readlines())
    print(f2.readlines())

['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']


In [5]:
with open('file1.txt') as f1:
    with open('file2.txt') as f2:
        with open('file3.txt') as f3:
            print(f1.readlines())
            print(f2.readlines())
            print(f3.readlines())

['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']


In [1]:
from contextlib import contextmanager

In [2]:
@contextmanager
def open_file(f_name):
    print(f'opening {f_name}')
    f = open(f_name)
    try:
        yield f
    finally:
        print(f'closing {f_name}')
        f.close()

In [5]:
f_names = 'file1.txt', 'file2.txt', 'file3.txt'
exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)